# Predicting P/E Ratios
**Brett Bartol**

In [89]:
import pandas as pd 
import numpy as np
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import xgboost as xgb
from regressors import stats
import warnings
warnings.filterwarnings('ignore')
import math
import os
import glob
import random
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
import statsmodels.formula.api as ols
import statsmodels.api as sm
from sklearn.impute import KNNImputer

### Importing & Cleaning the Data

In [41]:
data = pd.read_csv('Final_Assignment.csv')

In [42]:
data.head()

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,ajex,curcd,act,...,xintd,xopr,xrd,xsga,xt,costat,prcc_f,spcindcd,spcseccd,spcsrc
0,1000,1961/12/31,1961.0,INDL,C,D,STD,3.3418,USD,NaN,...,NaN,NaN,NaN,NaN,NaN,I,NaN,325.0,978.0,NaN
1,1000,1962/12/31,1962.0,INDL,C,D,STD,3.3418,USD,NaN,...,NaN,NaN,NaN,NaN,NaN,I,NaN,325.0,978.0,NaN
2,1000,1963/12/31,1963.0,INDL,C,D,STD,3.2445,USD,0.408,...,NaN,1.411,NaN,0.346,NaN,I,NaN,325.0,978.0,NaN
3,1000,1964/12/31,1964.0,INDL,C,D,STD,3.0900,USD,0.718,...,NaN,1.905,NaN,0.431,NaN,I,NaN,325.0,978.0,NaN
4,1000,1965/12/31,1965.0,INDL,C,D,STD,3.0900,USD,0.725,...,NaN,1.848,NaN,0.506,NaN,I,NaN,325.0,978.0,NaN


In [43]:
data.columns

Index(['gvkey', 'datadate', 'fyear', 'indfmt', 'consol', 'popsrc', 'datafmt',
       'ajex', 'curcd', 'act', 'ap', 'apalch', 'at', 'bkvlps', 'capx', 'ceq',
       'ch', 'chech', 'cogs', 'csho', 'dd1', 'dd2', 'dd3', 'dlc', 'dltt', 'dt',
       'dv', 'dvt', 'ebit', 'ebitda', 'fincf', 'invt', 'ivncf', 'lct', 'lt',
       'ni', 'nim', 'nopi', 'oancf', 'opeps', 'ppegt', 'ppent', 're', 'rect',
       'revt', 'sale', 'spce', 'wcap', 'wcapch', 'xacc', 'xad', 'xinst',
       'xint', 'xintd', 'xopr', 'xrd', 'xsga', 'xt', 'costat', 'prcc_f',
       'spcindcd', 'spcseccd', 'spcsrc'],
      dtype='object')

In [44]:
data = data.dropna(subset = ['prcc_f'], axis = 0)
data = data.dropna(subset = ['spce', 'spcindcd', 'spcseccd', 'spcsrc'], how = 'all')
data = data.drop(['spce', 'spcindcd', 'spcseccd', 'spcsrc', 'indfmt', 'consol', 'popsrc', 'datafmt', 'curcd', 'costat', 'wcapch'], axis = 1)

In [45]:
data['xad'] = data['xad'].fillna(0)
data['xrd'] = data['xrd'].fillna(0)
data['ajex'] = data.ajex.fillna(1)

data['fyear'] = data.fyear.fillna(data.datadate.apply(lambda x : int(str(x)[:4])))

In [46]:
data.isna().sum()

gvkey            0
datadate         0
fyear            0
ajex             0
act         113898
ap           62651
apalch      246095
at           26048
bkvlps       65731
capx         85328
ceq          28942
ch           63013
chech       143017
cogs         62912
csho         42398
dd1          64348
dd2         163239
dd3         164100
dlc          31933
dltt         29170
dt          260147
dv           99796
dvt          28497
ebit         72962
ebitda       69641
fincf       176359
invt         64034
ivncf       176358
lct         109661
lt           27882
ni           62160
nim         388543
nopi         63033
oancf       176418
opeps       166210
ppegt        86036
ppent        34348
re           34025
rect         71130
revt         28229
sale         62755
wcap        115642
xacc        180998
xad              0
xinst       396474
xint         68834
xintd       301700
xopr         62994
xrd              0
xsga        122902
xt          372519
prcc_f           0
dtype: int64

In [47]:
# imputer = KNNImputer(n_neighbors = 5, weights = 'uniform', metric = 'nan_euclidean')
# imputer.fit(data[data.columns[3:]])
# imputed_data = imputer.transform(data[data.columns[3:]].copy())

# data[data.columns[3:]] = imputed_data

# filling in missing values with median of the year
new_df = pd.DataFrame(columns = data.columns)
for i in range(2000, 2021):
    
    df2 = data[data['fyear'] == i]
    df2['act'] = df2['act'].fillna(df2['act'].median())
    df2['ap'] = df2['ap'].fillna(df2['ap'].median())
    df2['apalch'] = df2['apalch'].fillna(df2['apalch'].median())
    df2['at'] = df2['at'].fillna(df2['at'].median())
    df2['lct'] = df2['lct'].fillna(df2['lct'].median())
    df2['bkvlps'] = df2['bkvlps'].fillna(df2['bkvlps'].median())
    df2['capx'] = df2['capx'].fillna(df2['capx'].median())
    df2['ceq'] = df2['ceq'].fillna(df2['ceq'].median())
    df2['ch'] = df2['ch'].fillna(df2['ch'].median())
    df2['chech'] = df2['chech'].fillna(df2['chech'].median())
    df2['cogs'] = df2['cogs'].fillna(df2['cogs'].median())
    df2['dd1'] = df2['dd1'].fillna(df2['dd1'].median())
    df2['dd2'] = df2['dd2'].fillna(df2['dd2'].median())
    df2['dd3'] = df2['dd3'].fillna(df2['dd3'].median())
    df2['dlc'] = df2['dlc'].fillna(df2['dlc'].median())
    df2['dltt'] = df2['dltt'].fillna(df2['dltt'].median())
    df2['dt'] = df2['dt'].fillna(df2['dt'].median())
    df2['dv'] = df2['dv'].fillna(df2['dv'].median())
    df2['dvt'] = df2['dvt'].fillna(df2['dvt'].median())
    df2['ebit'] = df2['ebit'].fillna(df2['ebit'].median())
    df2['ebitda'] = df2['ebitda'].fillna(df2['ebitda'].median())
    df2['fincf'] = df2['fincf'].fillna(df2['fincf'].median())
    df2['ivncf'] = df2['ivncf'].fillna(df2['ivncf'].median())
    df2['invt'] = df2['invt'].fillna(df2['invt'].median())
    df2['lt'] = df2['lt'].fillna(df2['lt'].median())
    df2['ni'] = df2['ni'].fillna(df2['ni'].median())
    df2['nim'] = df2['nim'].fillna(df2['nim'].median())
    df2['nopi'] = df2['nopi'].fillna(df2['nopi'].median())
    df2['oancf'] = df2['oancf'].fillna(df2['oancf'].median())
    df2['opeps'] = df2['opeps'].fillna(df2['opeps'].median())
    df2['ppegt'] = df2['ppegt'].fillna(df2['ppegt'].median())
    df2['ppent'] = df2['ppent'].fillna(df2['ppent'].median())
    df2['re'] = df2['re'].fillna(df2['re'].median())
    df2['rect'] = df2['rect'].fillna(df2['rect'].median())
    df2['revt'] = df2['revt'].fillna(df2['revt'].median())
    df2['sale'] = df2['sale'].fillna(df2['sale'].median())
    df2['wcap'] = df2['wcap'].fillna(df2['wcap'].median())
    df2['xacc'] = df2['xacc'].fillna(df2['xacc'].median())
    df2['xad'] = df2['xad'].fillna(df2['xad'].median())
    df2['xinst'] = df2['xinst'].fillna(df2['xinst'].median())
    df2['xint'] = df2['xint'].fillna(df2['xint'].median())
    df2['xintd'] = df2['xintd'].fillna(df2['xintd'].median())
    df2['xopr'] = df2['xopr'].fillna(df2['xopr'].median())
    df2['xrd'] = df2['xrd'].fillna(df2['xrd'].median())
    df2['xsga'] = df2['xsga'].fillna(df2['xsga'].median())
    df2['xt'] = df2['xt'].fillna(df2['xt'].median())
    
    new_df = new_df.append(df2)
    
    new_df['pm'] = new_df['ni'] / new_df['revt']
    new_df['atr'] = new_df['revt'] / new_df['at']
    new_df['fl'] = new_df['at'] / new_df['ceq']
    new_df['cr'] = new_df['act'] / new_df['lct']
    new_df['de'] = new_df['dt'] / new_df['ceq']
    new_df['roa'] = new_df['ni'] / new_df['at']
    
    
data = new_df.copy()

In [48]:
data.columns

Index(['gvkey', 'datadate', 'fyear', 'ajex', 'act', 'ap', 'apalch', 'at',
       'bkvlps', 'capx', 'ceq', 'ch', 'chech', 'cogs', 'csho', 'dd1', 'dd2',
       'dd3', 'dlc', 'dltt', 'dt', 'dv', 'dvt', 'ebit', 'ebitda', 'fincf',
       'invt', 'ivncf', 'lct', 'lt', 'ni', 'nim', 'nopi', 'oancf', 'opeps',
       'ppegt', 'ppent', 're', 'rect', 'revt', 'sale', 'wcap', 'xacc', 'xad',
       'xinst', 'xint', 'xintd', 'xopr', 'xrd', 'xsga', 'xt', 'prcc_f', 'pm',
       'atr', 'fl', 'cr', 'de', 'roa'],
      dtype='object')

In [49]:
data

,gvkey,datadate,fyear,ajex,act,ap,apalch,at,bkvlps,capx,...,xrd,xsga,xt,prcc_f,pm,atr,fl,cr,de,roa
83,1004,2001/05/31,2000.0,1.0000,485.856,73.975,0.378,701.854,12.6299,13.134,...,0.000,96.077,74.322,14.000,0.021196,1.245637,2.062990,3.874697,0.530249,0.026403
260,1013,2000/10/31,2000.0,0.1429,2650.900,211.300,306.300,3970.500,3.7813,375.300,...,338.000,1050.700,74.322,21.375,0.264029,0.828082,1.363168,2.545760,0.015450,0.218637
393,1019,2000/12/31,2000.0,1.0000,8.840,2.235,0.378,28.638,79.4217,1.870,...,0.000,12.554,74.322,183.000,0.041455,1.484182,2.172178,1.640074,0.133874,0.061527
443,1021,2000/06/30,2000.0,0.0020,7.884,1.165,-0.292,11.608,0.4805,0.139,...,0.477,8.189,74.322,0.490,-0.031852,2.185303,2.605612,3.460931,1.138047,-0.069607
603,1034,2000/12/31,2000.0,1.0000,600.418,72.866,30.069,1610.435,21.0844,72.088,...,43.276,276.064,74.322,43.875,0.061621,0.559348,1.899351,2.908466,0.619329,0.034468
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588685,285135,2020/12/31,2020.0,1.0000,22.281,4.758,0.089,86.441,13.8467,18.558,...,0.000,3.806,356.802,5.710,-1.478091,0.135966,2.053523,1.705788,0.881693,-0.200969
588861,287462,2020/12/31,2020.0,1.0000,342.605,13.437,-48.910,441.304,8.5015,4.464,...,2.988,8.968,356.802,12.700,0.227691,0.463411,1.186674,10.385746,0.003845,0.105515
588926,290862,2020/09/30,2020.0,0.1000,27.542,0.128,-0.137,27.542,0.1105,0.000,...,0.000,1.524,356.802,0.085,inf,0.000000,1.164616,26.533719,0.000000,0.240868
589034,294524,2020/12/31,2020.0,1.0000,11634.000,2948.000,217.000,35403.000,23.8642,1947.000,...,113.000,1216.000,356.802,91.660,0.051166,0.783917,4.441475,2.114504,2.110902,0.040110


In [50]:
data.isna().sum()

gvkey           0
datadate        0
fyear           0
ajex            0
act             0
ap              0
apalch          0
at              0
bkvlps          0
capx            0
ceq             0
ch              0
chech           0
cogs            0
csho        22254
dd1             0
dd2             0
dd3             0
dlc             0
dltt            0
dt              0
dv              0
dvt             0
ebit            0
ebitda          0
fincf           0
invt            0
ivncf           0
lct             0
lt              0
ni              0
nim             0
nopi            0
oancf           0
opeps           0
ppegt           0
ppent           0
re              0
rect            0
revt            0
sale            0
wcap            0
xacc            0
xad             0
xinst           0
xint            0
xintd           0
xopr            0
xrd             0
xsga            0
xt              0
prcc_f          0
pm             45
atr           520
fl             44
cr        

In [60]:
data

,gvkey,datadate,fyear,ajex,act,ap,apalch,at,bkvlps,capx,...,xrd,xsga,xt,prcc_f,pm,atr,fl,cr,de,roa
83,1004,2001/05/31,2000.0,1.0000,485.856,73.975,0.378,701.854,12.6299,13.134,...,0.000,96.077,74.322,14.000,0.021196,1.245637,2.062990,3.874697,0.530249,0.026403
260,1013,2000/10/31,2000.0,0.1429,2650.900,211.300,306.300,3970.500,3.7813,375.300,...,338.000,1050.700,74.322,21.375,0.264029,0.828082,1.363168,2.545760,0.015450,0.218637
393,1019,2000/12/31,2000.0,1.0000,8.840,2.235,0.378,28.638,79.4217,1.870,...,0.000,12.554,74.322,183.000,0.041455,1.484182,2.172178,1.640074,0.133874,0.061527
443,1021,2000/06/30,2000.0,0.0020,7.884,1.165,-0.292,11.608,0.4805,0.139,...,0.477,8.189,74.322,0.490,-0.031852,2.185303,2.605612,3.460931,1.138047,-0.069607
603,1034,2000/12/31,2000.0,1.0000,600.418,72.866,30.069,1610.435,21.0844,72.088,...,43.276,276.064,74.322,43.875,0.061621,0.559348,1.899351,2.908466,0.619329,0.034468
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588685,285135,2020/12/31,2020.0,1.0000,22.281,4.758,0.089,86.441,13.8467,18.558,...,0.000,3.806,356.802,5.710,-1.478091,0.135966,2.053523,1.705788,0.881693,-0.200969
588861,287462,2020/12/31,2020.0,1.0000,342.605,13.437,-48.910,441.304,8.5015,4.464,...,2.988,8.968,356.802,12.700,0.227691,0.463411,1.186674,10.385746,0.003845,0.105515
588926,290862,2020/09/30,2020.0,0.1000,27.542,0.128,-0.137,27.542,0.1105,0.000,...,0.000,1.524,356.802,0.085,inf,0.000000,1.164616,26.533719,0.000000,0.240868
589034,294524,2020/12/31,2020.0,1.0000,11634.000,2948.000,217.000,35403.000,23.8642,1947.000,...,113.000,1216.000,356.802,91.660,0.051166,0.783917,4.441475,2.114504,2.110902,0.040110


In [84]:
df = data.copy()


# df = df[df['at'] != 0]
# df = df[df['lct'] != 0]
# df = df[df['act'] != 0]


df['ajex'] = df['ajex'].replace(0, 1)
df['ajex'] = df['ajex'].fillna(1)
df['csho_adj'] = df['csho'] / df['ajex']

df['prcc_f_adj'] = df['prcc_f'] * df['ajex']

df['pe'] = df.prcc_f_adj / (df.ebitda / df.csho_adj)

# converting variables to per share
df['actps'] = df['act'] / df['csho_adj']
df['apps'] = df['ap'] / df['csho_adj']
df['apalchps'] = df['apalch'] / df['csho_adj']
df['atps'] = df['at'] / df['csho_adj']
df['lctps'] = df['lct'] / df['csho_adj']
df['capxps'] = df['capx'] / df['csho_adj']
df['ceqps'] = df['ceq'] / df['csho_adj']
df['chps'] = df['ch'] / df['csho_adj']
df['chechps'] = df['chech'] / df['csho_adj']
df['cogsps'] = df['cogs'] / df['csho_adj']
df['dd1ps'] = df['dd1'] / df['csho_adj']
df['dd2ps'] = df['dd2'] / df['csho_adj']
df['dd3ps'] = df['dd3'] / df['csho_adj']
df['dlcps'] = df['dlc'] / df['csho_adj']
df['dlttps'] = df['dltt'] / df['csho_adj']
df['dtps'] = df['dt'] / df['csho_adj']
df['dvps'] = df['dv'] / df['csho_adj']
df['dvtps'] = df['dvt'] / df['csho_adj']
df['ebitps'] = df['ebit'] / df['csho_adj']
df['ebitdaps'] = df['ebitda'] / df['csho_adj']
df['fincfps'] = df['fincf'] / df['csho_adj']
df['invtps'] = df['invt'] / df['csho_adj']
df['ivncfps'] = df['ivncf'] / df['csho_adj']
df['ltps'] = df['lt'] / df['csho_adj']
df['nips'] = df['ni'] / df['csho_adj']
df['nopips'] = df['nopi'] /df['csho_adj']
df['oancfps'] = df['oancf'] / df['csho_adj']
df['ppegtps'] = df['ppegt'] / df['csho_adj']
df['ppentps'] = df['ppent'] /df['csho_adj']
df['rectps'] = df['rect'] / df['csho_adj']
df['revtps'] = df['revt'] / df['csho_adj']
df['saleps'] = df['sale'] / df['csho_adj']
df['wcapps'] = df['wcap'] / (df['csho_adj'])
#df['wcapchps'] = df['wcapch'] / (df['csho_adj'])
df['xaccps'] = df['xacc'] / df['csho_adj']
df['xadps'] = df['xad'] / df['csho_adj']
df['xintps'] = df['xint'] / df['csho_adj']
df['xoprps'] = df['xopr'] / df['csho_adj']
df['xrdps'] = df['xrd'] / df['csho_adj']
df['xsgaps'] = df['xsga'] / df['csho_adj']
df['xtps'] = df['xt'] / df['csho_adj']
df['reps'] = df['re'] / df['csho_adj']


df['pe_ny'] = df.pe.shift(-1)


df['PEchange'] = df.pe_ny - df.pe

df = df.sort_values(by = ['gvkey', 'fyear'], ascending = True)
df = df[df.gvkey == df.gvkey.shift(-1)]

for col in df.columns:
    m = df.loc[df[col] != np.inf, col].max()
    m2 = df.loc[df[col] != -np.inf, col].min()
    df[col].replace([np.inf], m, inplace = True)
    df[col].replace([-np.inf], m2, inplace = True)
    
df = df.dropna()

**Sentiment Data**

In [103]:
# reading in sentiment data

path = os.getcwd()
csv_files = glob.glob(os.path.join(path + '/filing_sentiments/', "*.csv"))

sentiment_data = pd.DataFrame()

for f in csv_files:
    csv = pd.read_csv(f)
    sentiment_data = sentiment_data.append(csv)   
    
sentiment_data = sentiment_data[sentiment_data['report_type'] == '10-K'].drop_duplicates()

In [104]:
sentiment_data['year'] = sentiment_data.date.apply(lambda x : int(x[:4]))
sentiment_data.year.value_counts()

2013    153
2012    139
2014    139
2015    136
2016    131
2017    119
2019    107
2018    103
2011    102
2020     98
2002     98
2021     95
2001     85
2003     73
2004     68
2007     60
2005     55
2010     53
2009     47
2008     44
2000     37
2006     36
2022      7
Name: year, dtype: int64

In [102]:
sentiment_data.date.value_counts()

2013-12-31    138
2014-12-31    132
2015-12-31    128
2012-12-31    122
2016-12-31    121
             ... 
2003-09-27      1
2019-03-30      1
2007-12-30      1
2015-01-31      1
2001-07-01      1
Name: date, Length: 360, dtype: int64

### Modelling

**LassoCV**

In [92]:
var = ['fyear', 'actps', 'apps', 'apalchps', 'atps', 'lctps', 'capxps', 'ceqps', 'chps', 'chechps', 'dd1ps',
      'dd2ps', 'dd3ps', 'dlcps', 'dlttps', 'dtps', 'dvps', 'dvtps', 'ebitps', 'ebitdaps', 'fincfps', 'invtps',
      'ivncfps', 'ltps', 'nips', 'nopips', 'oancfps', 'ppegtps', 'ppentps', 'rectps', 'revtps', 'saleps', 
      'wcapps', 'xaccps', 'xadps', 'xintps', 'xoprps', 'xrdps', 'xsgaps', 'xtps', 'reps', 'pe', 'pm', 'atr', 
      'fl', 'cr', 'de', 'roa']

In [93]:
results = []
yearly_coefs = []
lasso_rmse = []

for i in range(2011, 2020):
    train_range = range(i - 10, i)
    train_data = df[df['fyear'].isin(train_range)].copy()[var]
    train_labels = df[df['fyear'].isin(train_range)].copy()['PEchange']
    val_data = df[df['fyear'] == i].copy()[var]
    val_labels = df[df['fyear'] == i].copy()['PEchange']


    model_yearly = LassoCV(max_iter = 10000).fit(train_data, train_labels)
    
    print('\n', 'Year', i, 'RMSE: ', np.sqrt(mse(val_labels, model_yearly.predict(val_data))))
    print('\n', 'Year', i, 'R2: ', r2_score(val_labels, model_yearly.predict(val_data)))
    
    yearly_coefs.append(model_yearly.coef_)
    lasso_rmse.append(np.sqrt(mse(val_labels, model_yearly.predict(val_data))))
    results.append([i, model_yearly.predict(val_data), val_labels])


 Year 2011 RMSE:  4780.229632177161

 Year 2011 R2:  0.18999487712944474

 Year 2012 RMSE:  4409.216654190388

 Year 2012 R2:  0.31289908328707994

 Year 2013 RMSE:  5593.83327007318

 Year 2013 R2:  0.227128571730982

 Year 2014 RMSE:  5474.923776198573

 Year 2014 R2:  0.2649913397003957

 Year 2015 RMSE:  5592.344836891118

 Year 2015 R2:  0.19044102013328557

 Year 2016 RMSE:  5124.421833699398

 Year 2016 R2:  0.12826125921923837

 Year 2017 RMSE:  4549.694714881364

 Year 2017 R2:  0.16466995295958842

 Year 2018 RMSE:  2694.567733346362

 Year 2018 R2:  0.0010780727889279085

 Year 2019 RMSE:  2843.0186248554464

 Year 2019 R2:  0.03892392120173671


**Random Forrest**

In [91]:
results_rf = []
rf_rmse = []

for i in range(2010, 2020):
    train_range = range(i - 10, i)
    train_data = df[df['fyear'].isin(train_range)].copy()[var]
    train_labels = df[df['fyear'].isin(train_range)].copy()['PEchange']
    val_data = df[df['fyear'] == i].copy()[var]
    val_labels = df[df['fyear'] == i].copy()['PEchange']


    model_yearly = RandomForestRegressor().fit(train_data, train_labels)
    
    print('\n', 'Year', i, 'RMSE: ', np.sqrt(mse(val_labels, model_yearly.predict(val_data))))
    print('\n', 'Year', i, 'R2: ', r2_score(val_labels, model_yearly.predict(val_data)))
    
    
    rf_rmse.append(np.sqrt(mse(val_labels, model_yearly.predict(val_data))))
    results_rf.append([i, model_yearly.predict(val_data), val_labels])


 Year 2010 RMSE:  6545.224923593141

 Year 2010 R2:  0.2736417817789123

 Year 2011 RMSE:  4784.771937886993

 Year 2011 R2:  0.1884547674445265

 Year 2012 RMSE:  4371.702965212323

 Year 2012 R2:  0.3245410787243963

 Year 2013 RMSE:  5451.242375379703

 Year 2013 R2:  0.2660284915214617

 Year 2014 RMSE:  6065.349943058787

 Year 2014 R2:  0.09791382429666406

 Year 2015 RMSE:  5437.4782737111145

 Year 2015 R2:  0.23465776994357135

 Year 2016 RMSE:  5125.776582277172

 Year 2016 R2:  0.12780027338827726

 Year 2017 RMSE:  4964.677576717753

 Year 2017 R2:  0.005337628128540772

 Year 2018 RMSE:  3065.6879294484734

 Year 2018 R2:  -0.2930318553635769

 Year 2019 RMSE:  3188.754952348663

 Year 2019 R2:  -0.20903994727589326


**XGBoost**

In [94]:
results_xgb = []
xgb_rmse = []

for i in range(2010, 2020):
    train_range = range(i - 10, i)
    train_data = df[df['fyear'].isin(train_range)].copy()[var]
    train_labels = df[df['fyear'].isin(train_range)].copy()['PEchange']
    val_data = df[df['fyear'] == i].copy()[var]
    val_labels = df[df['fyear'] == i].copy()['PEchange']


    model_yearly = RandomForestRegressor().fit(train_data, train_labels)
    
    print('\n', 'Year', i, 'RMSE: ', np.sqrt(mse(val_labels, model_yearly.predict(val_data))))
    print('\n', 'Year', i, 'R2: ', r2_score(val_labels, model_yearly.predict(val_data)))
    
    
    xgb_rmse.append(np.sqrt(mse(val_labels, model_yearly.predict(val_data))))
    results_xgb.append([i, model_yearly.predict(val_data), val_labels])

KeyboardInterrupt: 

In [95]:
data.ni.describe()

count    179970.000000
mean        165.638718
std        1447.421439
min      -99289.000000
25%          -2.032000
50%           3.122000
75%          27.691750
max      104821.000000
Name: ni, dtype: float64

### Perfect Foresight Portfolio

In [131]:
fama = pd.read_csv('FFannual.csv')

fama = fama[fama['Year'].isin(range(2001, 2019))]
fama['Mkt-RF'] = fama['Mkt-RF'] / 100
fama['SMB'] = fama['SMB'] / 100
fama['HML'] = fama['HML'] / 100
fama['RF'] = fama['RF'] / 100

fama

,Year,Mkt-RF,SMB,HML,RF
74,2001,-0.1520,0.1824,0.1850,0.0383
75,2002,-0.2276,0.0445,0.0809,0.0165
76,2003,0.3075,0.2651,0.0511,0.0102
77,2004,0.1072,0.0443,0.0767,0.0120
78,2005,0.0309,-0.0236,0.0943,0.0298
79,2006,0.1060,0.0008,0.1175,0.0480
80,2007,0.0104,-0.0740,-0.1728,0.0466
81,2008,-0.3834,0.0245,0.0083,0.0160
82,2009,0.2826,0.0935,-0.0942,0.0010
83,2010,0.1737,0.1429,-0.0510,0.0012


In [158]:
price_df = df[['gvkey', 'fyear', 'prcc_f']].copy()
price_df['prco_f'] = price_df.prcc_f.shift(1)
price_df['ret'] = (price_df.prcc_f - price_df.prco_f) / price_df.prco_f
price_df = price_df[price_df.gvkey == price_df.gvkey.shift(1)]

#price_df = price_df[price_df['ret'] < 4]

perfect_foresight_ret = []
for i in range(2001, 2019):
    df_year = df[df['fyear'] == i]
    df_year = df_year.sort_values(by = ['PEchange'], ascending = False)
    
    num_grab = int(len(df_year) * .2)
    long_comps = df_year.gvkey[:num_grab]
    short_comps = df_year.gvkey[-num_grab:]
    
    year_ret = price_df[price_df['fyear'] == i]
    
    long_ret = year_ret[year_ret['gvkey'].isin(long_comps)]['ret'].mean()
    short_ret = year_ret[year_ret['gvkey'].isin(short_comps)]['ret'].mean()
    
    perfect_foresight_ret.append(long_ret - short_ret)

In [159]:
perfect_foresight_ret

[1.6279828914679975,
 1.580590802159676,
 2.7825909227854546,
 3.3882705030784344,
 0.7543459498928287,
 5.688204003531118,
 -0.79541707438386,
 -3.528593737385783,
 -0.018059664595650782,
 0.5640411381493026,
 0.581045029436766,
 1.7386057276622426,
 -5.256972011523903,
 0.9703181492554016,
 1.9165174041726483,
 20.724776826698168,
 18.86069503318143,
 0.5457332816602498]

In [160]:
y = perfect_foresight_ret
x = fama['Mkt-RF'].copy().dropna()
x = sm.add_constant(x)

model = sm.OLS(y, x)
results = model.fit()

alpha = results.params[0]
alpha = float(alpha)
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.047
Model:                            OLS   Adj. R-squared:                 -0.012
Method:                 Least Squares   F-statistic:                    0.7936
Date:                Sun, 01 May 2022   Prob (F-statistic):              0.386
Time:                        21:25:35   Log-Likelihood:                -58.563
No. Observations:                  18   AIC:                             121.1
Df Residuals:                      16   BIC:                             122.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.4017      1.661      1.446      0.167      -1.119       5.923
Mkt-RF         7.5948      8.526      0.891      0.386     -10.479      25.668
==============================================================================
Omnibus:                       14.432   Durbin-Watson:                   1.131
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               12.243
Skew:                           1.613   Prob(JB):                      0.00220
Kurtosis:                       5.433   Cond. No.                         5.47
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [138]:
price_df.ret.describe()

count    1.074740e+05
mean              inf
std               NaN
min     -1.000000e+00
25%     -2.776789e-01
50%      1.062827e-02
75%      3.238792e-01
max               inf
Name: ret, dtype: float64